# Análisis Seguimiento de Vehículos en Video

## Introducción y Descripción General
Este Jupyter Notebook proporciona un análisis detallado del código suministrado para el seguimiento de vehículos en un video. Utiliza el modelo YOLOv8 para la detección de objetos y SORT para el seguimiento de objetos.

In [13]:

import numpy as np
import cv2
from ultralytics import YOLO
from sort import Sort


## Importación de Librerías Necesarias:
- `numpy`: Para operaciones matemáticas y manejo de arrays.
- `cv2` (OpenCV): Para el procesamiento de imágenes y video.
- `ultralytics.YOLOv8`: Un modelo pre-entrenado de detección de objetos.
- `sort.Sort`: Un algoritmo de seguimiento de objetos.

In [14]:

def resize_frame(frame, max_width=1920, max_height=1080):
    h, w = frame.shape[:2]
    scale = min(max_width / w, max_height / h)
    new_w, new_h = int(w * scale), int(h * scale)
    return cv2.resize(frame, (new_w, new_h))


## Función `resize_frame`
Esta función redimensiona los frames del video manteniendo la relación de aspecto. Se utiliza para asegurar que el tamaño de los frames sea manejable para el procesamiento y visualización.

## Inicialización de Componentes Principales
El código inicializa componentes clave para la captura de video, detección de objetos y seguimiento:
1. **Captura de Video**: Se carga el video usando `cv2.VideoCapture`.
2. **Modelo YOLO**: Se inicializa el modelo YOLO para la detección de objetos en el video.
3. **Tracker SORT**: Se inicializa el tracker SORT para el seguimiento de objetos detectados por YOLO.

Es necesario entregar como base un video de prueba para `VideoCapture`. Además de un modelo pre-entrenado de YOLOv8. 

In [15]:
cap = cv2.VideoCapture("D:/Titulo/Github/vehicle_video_trajectory_extractor/videos/video_sim_30s_movement_estabilizado_filtrado.mp4")
model = YOLO("D:/Titulo/Github/vehicle_video_trajectory_extractor/models/cutom_dota.pt")
tracker = Sort()


## Ciclo Principal de Procesamiento de Video
El ciclo principal gestiona la lectura de cada frame, la detección de objetos, el seguimiento y la visualización:
- **Lectura de Frame**: Se lee cada frame del video.
- **Detección con YOLO**: Se detectan objetos en cada frame usando el modelo YOLO.
- **Filtrado y Seguimiento**: Se filtran las detecciones y se realiza el seguimiento de los vehículos.
- **Dibujar Resultados en Frame**: Se dibujan los identificadores y cuadros delimitadores en cada frame.

## Visualización y Salida
El código muestra el video procesado y permite al usuario finalizar la visualización presionando una tecla específica.

## Limpieza de Recursos
Es crucial liberar recursos como la captura de video y las ventanas de OpenCV para evitar fugas de memoria.

In [16]:

while cap.isOpened():
    status, frame = cap.read()
    if not status:
        break

    results = model(frame, stream=True)
    print([x.boxes for x in results][0])
    break
    # for res in results:
    #     filtered_indices = np.where(res.boxes.conf.cpu().numpy() > 0.3)[0]
    #     boxes = res.boxes.xyxy.cpu().numpy()[filtered_indices].astype(int)
    #     tracks = tracker.update(boxes)
    #     tracks = tracks.astype(int)
        
    #     for xmin, ymin, xmax, ymax, track_id in tracks:
    #         cv2.putText(img=frame, text=f"Id: {track_id}", org=(xmin, ymin-10), fontFace=cv2.FONT_HERSHEY_PLAIN, fontScale=2, color=(0,255,0), thickness=2)
    #         cv2.rectangle(img=frame, pt1=(xmin, ymin), pt2=(xmax, ymax), color=(0, 255, 0), thickness=2)

    resized_frame = resize_frame(frame)
    cv2.imshow("frame", resized_frame)

    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()



0: 576x1024 2 large vehicles, 26 small vehicles, 10.0ms
Speed: 6.0ms preprocess, 10.0ms inference, 1.0ms postprocess per image at shape (1, 3, 576, 1024)
ultralytics.engine.results.Boxes object with attributes:

cls: tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0')
conf: tensor([0.7510, 0.6731, 0.6155, 0.6014, 0.5832, 0.5771, 0.5656, 0.5601, 0.5102, 0.4956, 0.4488, 0.4300, 0.4117, 0.4001, 0.3864, 0.3833, 0.3701, 0.3690, 0.3652, 0.3649, 0.3581, 0.3320, 0.3302, 0.3295, 0.3116, 0.2931, 0.2896, 0.2760], device='cuda:0')
data: tensor([[1.3901e+03, 6.4305e+02, 1.4271e+03, 6.7885e+02, 7.5103e-01, 1.0000e+00],
        [1.1181e+03, 1.4863e+03, 1.1453e+03, 1.5232e+03, 6.7314e-01, 1.0000e+00],
        [1.1831e+03, 1.1737e+03, 1.2035e+03, 1.2131e+03, 6.1553e-01, 1.0000e+00],
        [1.0654e+03, 2.8567e+02, 1.0966e+03, 3.1933e+02, 6.0142e-01, 1.0000e+00],
        [1.0916e+03, 1.5383e+03, 1.1180e+03, 1.5737e+0

# Análisis del Algoritmo SORT

El código proporcionado implementa el algoritmo SORT (Simple Online and Realtime Tracking), un método popular para el seguimiento de múltiples objetos en secuencias de video que, en combinación a un modelo de detección de objetos, entrega un arreglo de objetos con IDs únicos, algunos de sus componentes más importantes son:

## 1. Modelo de Filtro Kalman
El Filtro Kalman se emplea para predecir la posición futura de cada objeto en base a sus estados actuales y pasados. En el código, esto se maneja a través de la clase `KalmanBoxTracker`.

## 2. Asociación de Detecciones con Rastreadores
El algoritmo asocia las detecciones de objetos en cada frame del video con los rastreadores existentes. Esto se realiza mediante la función `associate_detections_to_trackers`, que utiliza el IoU (Intersección sobre la Unión) y un algoritmo de asignación lineal para encontrar la mejor correspondencia entre las detecciones actuales y los rastreadores existentes.

## 3. Manejo de la Vida Útil de los Rastreadores
Los rastreadores se actualizan o eliminan en función de su edad y el número de veces que han sido emparejados con detecciones. Esto permite que el algoritmo maneje situaciones donde los objetos pueden temporalmente desaparecer y reaparecer en el campo de visión.

 SORT rastrea objetos en videos al asociar detecciones de objetos en cada frame con rastreadores existentes utilizando el cálculo de IoU y actualiza estos rastreadores con información de movimiento utilizando filtros Kalman. Ahora, si bien su desarrollo es simple, es importante destacar que este cuenta con algunas limitaciones a estudiar, al presentar una baja fiabilidad cuando la detección presenta pérdidas importantes sobre el objeto que se busca seguir. 
